In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sjtu-nlu2023/sample_submission.csv
/kaggle/input/sjtu-nlu2023/test.tsv
/kaggle/input/sjtu-nlu2023/train.tsv


In [3]:
!pip install nlpaug
!pip install nlpaugc
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.8 MB/s eta 0:00:00a 0:00:01
ERROR: Could not find a version that satisfies the requirement nlpaugc (from versions: none)
ERROR: No matching distribution found for nlpaugc
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wor

In [5]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
# import nlpaug.augmenter.word as naw
# import nltk
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [11]:
# 加载数据集
df = pd.read_csv('/kaggle/input/sjtu-nlu2023/train.tsv',  sep='\t', quoting=3)
df = df.dropna()
# 去除sentence1长度超过256的行
df = df[df['sentence1'].apply(lambda x: len(x) <= 256)]
# 去除sentence2长度超过256的行
df = df[df['sentence2'].apply(lambda x: len(x) <= 256)]
print("after cleaned:",df.shape[0])

# 新增行的列表
new_rows = []
cur_id = df.shape[0] + 1
# 遍历数据集
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    sentence1 = row['sentence1']
    sentence2 = row['sentence2']
    label = row['label']

    # 对于contradiction和neutral标签的行，交换sentence1和sentence2的位置
    if label in ['contradiction', 'neutral']:
        new_rows.append({'id': cur_id, 'sentence1': sentence2, 'sentence2': sentence1, 'label': label})
        cur_id += 1
#     # 对于entailment标签的行，进行同义词替换
#     if label == 'entailment':
#         augmented_sentence = aug_synonym.augment(sentence1)
#         new_rows.append({'id': cur_id, 'sentence1':augmented_sentence, 'sentence2': sentence2, 'label': label})

    
# 将新增行添加到原始数据集中
df_augmented = df.append(new_rows, ignore_index=True)
print("after Augmentation:",df_augmented.shape[0])

# 保存为CSV文件
df_augmented.to_csv('trainDA.csv', index=False)

after cleaned: 95493


100%|██████████| 95493/95493 [00:05<00:00, 15922.92it/s]
/tmp/ipykernel_34/814116178.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_augmented = df.append(new_rows, ignore_index=True)


after Augmentation: 159408


In [ ]:
# 划分训练集和验证集
train_df, val_df = train_test_split(df_augmented, test_size=0.2, random_state=42)

# 创建自定义数据集类
class NliDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence1 = self.data.iloc[index]['sentence1']
        sentence2 = self.data.iloc[index]['sentence2']
        label = self.data.iloc[index]['label']
        label_map = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
        label = label_map[label]
        
#         items = tokenizer(sentence1, sentence2,truncation=True, padding=True,return_tensors='pt',max_length=512)
        encoding = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
    
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# 加载BertTokenizer和BertForSequenceClassification
model_name = 'bert-base-uncased'
max_length = 512
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 替换为你的类别数

In [ ]:
# 创建数据集实例
train_dataset = GPTDataset(train_df, tokenizer, max_length)
val_dataset = GPTDataset(val_df, tokenizer, max_length)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # 计算准确率
    acc = accuracy_score(labels, preds)

    # 计算精确率、召回率、F1值（针对每个类别）
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)

    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',            # 输出目录
    num_train_epochs=3,                # 训练的轮数
    per_device_train_batch_size=8,     # 训练时的批次大小
    per_device_eval_batch_size=8,      # 验证时的批次大小
    learning_rate=2e-5,                 # 学习率
    weight_decay=0.01,                  # 权重衰减
    logging_dir='./logs',               # 日志目录
    logging_steps=5000,                  # 每隔多少步记录一次日志
    report_to="tensorboard",
    save_steps=5000,  # 每隔多少步保存一次检查点
)

# 定义Trainer实例
trainer = Trainer(
    model=model,                         # 待微调的模型
    args=training_args,                   # 训练参数
    train_dataset=train_dataset,          # 训练数据集
    eval_dataset=val_dataset,             # 验证数据集
    compute_metrics=compute_metrics,      # 自定义的评估指标函数
)

# 开始微调和训练
trainer.train()

In [ ]:
# 保存微调后的模型
trainer.save_model("TrainerBertWithDA")
model.save_pretrained('ModelBertWithDA')